### Merge data
One dataframe that contains county-level data associated with HIV Incidence is created by merging data from:
* amfAR (The Foundation of AIDS Research)
* CAMP (Emory Coalition for Applied Modeling for Prevention)
* ACS (The American Community Survey)

This process is based on Eric Logue's [Forecasting HIV Infections](https://github.com/elogue01/Forecasting-HIV-Infections) capstone project.

In [1]:
import pandas as pd

### amfAR data

In [2]:
#load Amfar opioid and HIV data, add county code
opiod_df = pd.read_table('data/amfAR/countydata.tsv',header=0)
opiod_df['county_code'] = opiod_df.STATEFP*1000 + opiod_df.COUNTYFP # build a county code column
opiod_df['county_code'] = opiod_df.county_code.astype(int)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


FileNotFoundError: [Errno 2] File b'data/amfAR/countydata.tsv' does not exist: b'data/amfAR/countydata.tsv'

In [ ]:
opiod_df.head()

The INDICATOR and VALUE columns contain HIV related data.

In [ ]:
#make changes to the amfar dataframe
#convert from long to wide format
index_lst = ['county_code', 'COUNTY', 'STATEABBREVIATION', 'YEAR']
col_lst = ['INDICATOR']
opiod_df_wide = opiod_df.pivot_table(values='VALUE', index=index_lst, columns=col_lst).reset_index()

In [ ]:
opiod_df_wide.head()

In [ ]:
# Focus on just the HIV related data, from 2008 onward
opiod_df_wide = opiod_df_wide[opiod_df_wide['YEAR'] >= 2008] # subset for years that have hiv data
cols_to_drop = ['CDC_consult', 'vulnerable_rank']
#, 'num_SSPs', 'bup_phys', 'drugdep', 'pctunmetneed', 'nonmedpain']
opiod_df_wide.drop(cols_to_drop, axis=1, inplace=True) # drop unnecessary columns

In [ ]:
opiod_df_wide.head()

By default pandas doesn't show all the columns (shows ... instead).  Set max columns so that all can be inspected.

In [ ]:
pd.get_option("display.max_columns")

In [ ]:
pd.set_option("display.max_columns", 100)

In [ ]:
opiod_df_wide.head()

Many NaNs in the data.  Fill numerical where appropriate, per Eric.

In [ ]:
fill_nan_cols = ['HIVdiagnoses', 'HIVincidence', 'HIVprevalence',
                 'PLHIV', 'drugdeathrate', 'drugdeaths']
opiod_df_wide[fill_nan_cols] = opiod_df_wide[fill_nan_cols].fillna(0) #fill NaNs for suppressed data with zeroes

In [ ]:
opiod_df_wide.head()

In [ ]:
# Subset data to 2015
opiod_df_15 = opiod_df_wide[opiod_df_wide['YEAR'] == 2015]

In [ ]:
opiod_df_15.head()

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn', get rid of annoying warning associated with dropping below
# drop columns having no 2015 data, will be extraploated from following years later
opiod_df_15.drop(['num_SSPs', 'bup_phys', 'drugdep', 'pctunmetneed', 'nonmedpain'], axis=1, inplace=True)

In [ ]:
opiod_df_15.head()

In [ ]:
# get esimates for num_SSPs, bug_phys, drug_dep, pctunmetneed, and nonmedpain from following years

#subset opioid related data from one year only
#number of needle exchange programs (num_SSPs)
opiod_df_wide_17 = opiod_df_wide[opiod_df_wide['YEAR'] == 2017]
df_num_SSP = opiod_df_wide_17[['num_SSPs', 'county_code']]

#number of doctors licensed to rx Buprenorphine (bup_phys)
df_bup_phys = opiod_df_wide_17[['bup_phys', 'county_code']]

#percent with drug dependency (drug_dep)
opiod_df_wide_16 = opiod_df_wide[opiod_df_wide['YEAR'] == 2016]
df_drugdep = opiod_df_wide_16[['drugdep', 'county_code']]

#percent unmet drug treatment need (pctunmetneed)
df_pctunmetneed = opiod_df_wide_16[['pctunmetneed', 'county_code']]

#percent taken pain meds for nonmedical use (nonmedpain)
df_nonmedpain = opiod_df_wide_16[['nonmedpain', 'county_code']]

In [ ]:
# merge these values back into 2015 dataframe
#merge opioid related data back to the 2015 dataframe
opiod_df_15 = opiod_df_15.merge(df_num_SSP, on='county_code')
opiod_df_15 = opiod_df_15.merge(df_bup_phys, on='county_code')
opiod_df_15 = opiod_df_15.merge(df_drugdep, on='county_code')
opiod_df_15 = opiod_df_15.merge(df_pctunmetneed, on='county_code')
opiod_df_15 = opiod_df_15.merge(df_nonmedpain, on='county_code')

In [ ]:
opiod_df_15.head()

### CAMP data

In [ ]:
#load Men who have sex with men (MSM) estimate data
msm_df = pd.read_csv("data/CAMP/US_MSM_Estimates_Data_2013.csv")    #load the data
msm_df['county_code'] = msm_df.STATEFP*1000 + msm_df.COUNTYFP  # build a county code column
msm_df['county_code'] = msm_df.county_code.astype(int)
msm_df['%msm12month'] = 100 * (msm_df.MSM12MTH / msm_df.ADULTMEN) # build a %MSM within last 12 months column
msm_df['%msm5yr'] = 100 * (msm_df.MSM5YEAR / msm_df.ADULTMEN)     # build a %MSM within last 5years column

In [ ]:
msm_df.head()

In [ ]:
cols_to_drop = ['REGCODE', 'DIVCODE', 'STATEFP', 'COUNTYFP', 'CSACODE', 
                'CBSACODE','METDCODE', 'METMICSA', 'CENTOUTL']
msm_df.drop(cols_to_drop, axis=1, inplace=True) #drop all unneeded columns
msm_df.head()

### ACS data

In [ ]:
#unemplyment data
df_employment = pd.read_csv("data/ACS_14_5YR_employment/ACS_14_5YR_S2301_with_ann.csv", 
                            encoding = "ISO-8859-1", skiprows=1)
df_employment = df_employment[['Id2', 'Unemployment rate; Estimate; Population 16 years and over']]
df_employment.columns = ['county_code', 'unemployment_rate']
df_employment.head()

In [ ]:
#poverty data
df_poverty = pd.read_csv("data/ACS_14_5YR_poverty/ACS_14_5YR_S1701_with_ann.csv", 
                         encoding = "ISO-8859-1", skiprows=1)
df_poverty = df_poverty[['Id2', 'Percent below poverty level; Estimate; Population for whom poverty status is determined']]
df_poverty.columns = ['county_code', 'poverty_rate']
df_poverty.head()

In [ ]:
#income data
df_income = pd.read_csv("data/ACS_14_5YR_income/ACS_14_5YR_S1901_with_ann.csv", 
                        encoding = "ISO-8859-1", skiprows=1)
df_income = df_income[['Id2', 'Households; Estimate; Total']]
df_income.columns = ['county_code', 'household_income']
df_income.head()

**Note:** Insurance data is also included in `data/ACS_14_5Y_health_insur_coverage` but 
reading and cleaning the data is left to the user, if desired.

### Merging all the dataframes together into one dataframe

In [ ]:
#merge asfAR hiv/opioid data with CAMP MSM data
df_main = opiod_df_15.merge(msm_df, on='county_code')

#merge in ACS data
df_main = df_main.merge(df_employment, on='county_code')
df_main = df_main.merge(df_poverty, on='county_code')
df_main = df_main.merge(df_income, on='county_code')

In [ ]:
df_main.info()

In [ ]:
df_main.head()

In [ ]:
import pandas as pd
pd.plotting.scatter_matrix(df_main);

In [ ]:
df_small = df_main[['HIVincidence','household_income','poverty_rate','unemployment_rate','MSM12MTH','AMAT_fac']]

In [ ]:
pd.plotting.scatter_matrix(df_small);

In [ ]:
df_county_study = df_main.groupby(['COUNTY','poverty_rate'])['HIVincidence'].mean()
df_county_study.plot(kind='bar')

In [ ]:
#poverty rate

import matplotlib.pyplot as plt
fig,ax = plt.subplots()

ax.scatter(x=df_main['poverty_rate'],y=df_main['HIVincidence'])
ax.set_xlabel('poverty_rate')
ax.set_ylabel('HIVincidence')
ax.set_ylim((0, 140))

In [ ]:
#unemployment_rate

fig,ax = plt.subplots()
ax.scatter(x=df_main['unemployment_rate'],y=df_main['HIVincidence'])
ax.set_xlabel('unemployment_rate')
ax.set_ylabel('HIVincidence')
ax.set_ylim((0, 130))

In [ ]:
#household income

import matplotlib.pyplot as plt
fig,ax = plt.subplots()
ax.scatter(x=df_main['household_income'],y=df_main['HIVincidence'])
ax.set_xlabel('household_income')
ax.set_ylabel('HIVincidence')
ax.set_ylim((0, 150))

In [ ]:
#drug dependency

import matplotlib.pyplot as plt
fig,ax = plt.subplots()
ax.scatter(x=df_main['drugdep'],y=df_main['HIVincidence'])
ax.set_xlabel('drugdep')
ax.set_ylabel('HIVincidence')
ax.set_ylim((0, 150))



In [ ]:
import statsmodels.api as sm


In [ ]:
y = df_main['HIVincidence']